# Проект: Анализ базы данных книжного сервиса.SQL

## Описание проекта


### Цели проекта

Компания приобрела крупный сервис для чтения книг по подписке.  

Необходимо проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

Эти данные помогут сформулировать ценностное предложение для нового продукта.  

### Описание данных

Таблица **books**

Содержит данные о книгах:  
* `book_id` — идентификатор книги;  
* `author_id` — идентификатор автора;  
* `title` — название книги;  
* `num_pages` — количество страниц;  
* `publication_date` — дата публикации книги;
* `publisher_id` — идентификатор издателя.  

---

Таблица **authors**  

Содержит данные об авторах:  
* `author_id` — идентификатор автора;  
* `author` — имя автора.  

---

Таблица **publishers**

Содержит данные об издательствах:  
* `publisher_id` — идентификатор издательства;  
* `publisher` — название издательства;  

---

Таблица **ratings**  

Содержит данные о пользовательских оценках книг:  
* `rating_id` — идентификатор оценки;      
* `book_id` — идентификатор книги;
* `username` — имя пользователя, оставившего оценку;  
* `rating` — оценка книги.  

---

Таблица **reviews**

Содержит данные о пользовательских обзорах на книги:  
* `review_id` — идентификатор обзора;  
* `book_id` — идентификатор книги;    
* `username` — имя пользователя, написавшего обзор;  
* `text` — текст обзора.  

---

### Задачи проекта

В рамках проекта будут решены следующие задачи:

* Загрузим библиотеки и проверим существующие таблицы;
* Посчитаем, сколько книг вышло после 1 января 2000 года;  
* Для каждой книги посчитаем количество обзоров и среднюю оценку;
* Определим издательство, которое выпустило наибольшее число книг толще
50 страниц — так мы исключим из анализа брошюры;
* Определим автора с самой высокой средней оценкой книг, учитывая
только книги с 50 и более оценками;
* Посчитаем среднее количество обзоров от пользователей, которые поставили
больше 48 оценок.
* Опишием выводы по каждой из решённых задач.

## Загрузка библиотек и обзор данных

### Загрузка библиотек

In [1]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa

In [ ]:
# устанавливаем параметры
db_config = {
'user': '...', # имя пользователя
'pwd': '...', # пароль
'host': '...',
'port': 6432, # порт подключения
'db': '...' # название базы данных
}

connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
# чтобы выполнять SQL-запросы, напишем функцию с использованием Pandas

def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
  '''Открываем соединение, получаем данные из sql, закрываем соединение'''
  with engine.connect() as con:
    return pd.read_sql(sql=sa.text(query), con = con)

### Обзор данных

In [3]:
# Выведем таблицы для обзора
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']

for i in tables:
  query = f'''
  SELECT *
  FROM {i}
  '''
  print(f'\n\nОбзор таблицы {i}\n{20*"*"}')
  display(get_sql_data(query))



Обзор таблицы books
********************


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331




Обзор таблицы authors
********************


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston




Обзор таблицы publishers
********************


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling




Обзор таблицы ratings
********************


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2




Обзор таблицы reviews
********************


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


Загружены библиотеки и произведен обзор доступных таблиц.  

База данных содержит 5 таблиц:  
* `books` - содержит информацию о 1000 книгах
* `authors` - список из 636 авторов
* `publishers` - в таблице перечень из 340 издательств
* `reviews` - содержит записи о 2793 отзывах
* `ratings` - 6456 строк с оценками пользователей

Таблицы имеют связи и подходят для решения поставленных задач.

## Решения

### Задача №1

Посчитаем, сколько книг вышло после 1 января 2000 года

In [4]:
query = '''
  SELECT COUNT(DISTINCT book_id)
  FROM books
  WHERE publication_date > '2000-01-01'
  '''
result = get_sql_data(query)
display(result)

,count
0,819


&ensp; После 1 января 2000 года было выпущено 819 книг.

---

### Задача №2

Для каждой книги посчитаем количество обзоров и среднюю оценку

In [5]:
query = '''
  SELECT b.book_id,
          b.title,
          COUNT(DISTINCT rw.review_id) AS COUNT_reviews,
          ROUND(AVG(rt.rating),2) AS AVG_rating
  FROM books as b

  LEFT JOIN ratings AS rt ON b.book_id=rt.book_id
  LEFT JOIN reviews as rw ON b.book_id=rw.book_id
  GROUP BY b.book_id, b.title
  ORDER BY count_reviews DESC
  '''
result = get_sql_data(query)
display(result)

,book_id,title,count_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,963,Water for Elephants,6,3.98
2,734,The Glass Castle,6,4.21
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,695,The Curious Incident of the Dog in the Night-Time,6,4.08
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
996,808,The Natural Way to Draw,0,3.00
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,221,Essential Tales and Poems,0,4.00


&ensp; Для каждой книги расчитано количество обзоров и средний рейтинг.

&ensp; Максимальное количество обзоров у книги Twilight - 7шт., имеющей средний рейтинг 3.66

---

### Задача №3

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [6]:
query = '''
  WITH t AS (
    SELECT p.publisher_id AS p_publisher_id,
          p.publisher AS p_publisher
    FROM books AS b
    LEFT JOIN publishers AS p ON b.publisher_id=p.publisher_id
    WHERE b.num_pages > 50
  )

  SELECT p_publisher AS Издательство,
        COUNT(p_publisher_id) AS "Кол-во книг"
  FROM t
  GROUP BY p_publisher_id, p_publisher
  ORDER BY COUNT(p_publisher_id) DESC
  LIMIT 1;

  '''
result = get_sql_data(query)
display(result)

,Издательство,Кол-во книг
0,Penguin Books,42


Наибольшее число книг с количеством страниц более 50 выпустило издательство Penguin Books - 42шт.

---

### Задача №4

Определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

In [7]:
query = '''
SELECT author AS Автор, AVG(avg_rate) AS "Средний рейтинг"
FROM books AS b

INNER JOIN(
  SELECT book_id, COUNT(rating_id) AS "Кол-во оценок", ROUND(AVG(rating),2) AS "avg_rate"
  FROM ratings
  GROUP BY book_id
  HAVING COUNT(rating_id) >=50 ) AS rt ON rt.book_id=b.book_id

LEFT JOIN authors AS a ON a.author_id=b.author_id

GROUP BY author
ORDER BY "Средний рейтинг" DESC
LIMIT 1;

  '''
result = get_sql_data(query)
display(result)

,Автор,Средний рейтинг
0,J.K. Rowling/Mary GrandPré,4.285


Самый высокий средний рейтинг книг с количеством оценок от 50шт -  
у автора J.K. Rowling (в соавторстве с иллюстратором Mary GrandPré)

---

### Задача №5

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [8]:
query = '''

WITH t AS (
  SELECT username, COUNT(review_id) AS count_review_ids
    FROM books AS b
  LEFT JOIN reviews AS rw ON b.book_id = rw.book_id
  WHERE rw.username IN (
      SELECT username
      FROM ratings
      GROUP BY username
      HAVING COUNT(rating_id) > 48)

  GROUP BY username
  )

SELECT AVG(count_review_ids)
FROM t;

  '''
result = get_sql_data(query)
display(result)

,avg
0,24.0


В среднем пользователи, оставившие более 48 оценок, написали по 24 обзора.

---

### Заключение

В рамках проекта была изучена база данных сервиса для чтения книг по подписке.

База данных состоит из 5 таблиц и содержит информацию о загруженных книгах, издательствах, авторах, а также об оценках и отзывах пользователей.

В процессе исследования были решены поставленные задачи:

* После 1 января 2000 года было выпущено 819 книг.

* Максимальное количество обзоров у книги Twilight - 7шт., имеющей средний рейтинг 3.66  

* Наибольшее число книг с количеством страниц более 50 выпустило издательство Penguin Books - 42шт.

* Самый высокий средний рейтинг книг с количеством оценок от 50шт -  
у автора J.K. Rowling (в соавторстве с иллюстратором Mary GrandPré)

* В среднем пользователи, оставившие более 48 оценок, написали по 24 обзора.  
<br></br>


---
---
---